<a href="https://colab.research.google.com/github/Thilan67/Check_similarity_in_two_images/blob/main/Image_Similarity_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install flask pillow opencv-python scikit-image numpy pyngrok
from pyngrok import ngrok
from getpass import getpass

ngrok_token = getpass("Enter your ngrok token: ")
ngrok.set_auth_token(ngrok_token)


from flask import Flask, request, render_template_string
from PIL import Image
import cv2
import numpy as np
from skimage.metrics import structural_similarity

app = Flask(__name__)


def my_algorithm(image1, image2):

    # Convert PIL Images to OpenCV format
    img1_array = np.array(image1)
    img2_array = np.array(image2)

    # Convert RGB to BGR (OpenCV format)
    image1_cv = cv2.cvtColor(img1_array, cv2.COLOR_RGB2BGR)
    image2_cv = cv2.cvtColor(img2_array, cv2.COLOR_RGB2BGR)

    # Resize images to same size if different
    if image1_cv.shape != image2_cv.shape:
        # Resize image2 to match image1 dimensions
        image2_cv = cv2.resize(image2_cv, (image1_cv.shape[1], image1_cv.shape[0]))

    # Convert to grayscale
    gray_orig = cv2.cvtColor(image1_cv, cv2.COLOR_BGR2GRAY)
    gray_mod = cv2.cvtColor(image2_cv, cv2.COLOR_BGR2GRAY)

    # Calculate structural similarity
    (score, diff) = structural_similarity(gray_orig, gray_mod, full=True)
    diff = (diff * 255).astype("uint8")

    # Calculate Mean Squared Error
    mse = np.mean((image1_cv - image2_cv) ** 2)

    # ORB Feature Matching
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(image1_cv, None)
    kp2, des2 = orb.detectAndCompute(image2_cv, None)

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(des1, des2)
    feature_similarity = len(matches)

    # Histogram Comparison (Color Similarity)
    hist1 = cv2.calcHist([image1_cv], [0], None, [256], [0, 256])
    hist2 = cv2.calcHist([image2_cv], [0], None, [256], [0, 256])
    color_similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

    # Return all results
    result = {
        "Structural Similarity Index (SSIM)": f"{score:.4f}",
        "Pixel Difference (MSE)": f"{mse:.2f}",
        "Color Similarity Score": f"{color_similarity:.4f}",
        "Feature-based Similarity": f"{feature_similarity}"
    }

    return result


Enter your ngrok token: ··········


In [5]:
HTML = '''
<!DOCTYPE html>
<html>
<head>
    <title>Image Similarity Checker</title>
    <style>
        body {
            font-family: Arial;
            max-width: 900px;
            margin: 50px auto;
            padding: 20px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        }
        .container {
            background: white;
            padding: 40px;
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        }
        h1 {
            text-align: center;
            color: #333;
            margin-bottom: 30px;
        }
        .upload-section {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 20px;
            margin-bottom: 20px;
        }
        .upload-box {
            padding: 20px;
            border: 3px dashed #667eea;
            border-radius: 10px;
            text-align: center;
            background: #f8f9ff;
        }
        .upload-box h3 {
            color: #667eea;
            margin-bottom: 15px;
        }
        input[type="file"] {
            margin: 10px 0;
            width: 100%;
        }
        button {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 15px 40px;
            border: none;
            border-radius: 30px;
            font-size: 18px;
            font-weight: bold;
            cursor: pointer;
            display: block;
            margin: 30px auto;
            transition: transform 0.2s;
        }
        button:hover {
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4);
        }
        .result {
            margin-top: 30px;
            padding: 30px;
            background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
            border-radius: 10px;
        }
        .result h2 {
            text-align: center;
            color: #333;
            margin-bottom: 20px;
        }
        .result-item {
            padding: 15px;
            margin: 10px 0;
            background: white;
            border-radius: 8px;
            display: flex;
            justify-content: space-between;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }
        .result-label {
            font-weight: bold;
            color: #667eea;
        }
        .result-value {
            color: #333;
            font-size: 24px;
            font-weight: bold;
        }
        @media (max-width: 768px) {
            .upload-section {
                grid-template-columns: 1fr;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Image Similarity Checker</h1>
        <p style="text-align: center; color: #666; margin-bottom: 30px;">
            Upload two images to compare their similarity
        </p>

        <form method="POST" enctype="multipart/form-data">
            <div class="upload-section">
                <div class="upload-box">
                    <h3>📷 Image 1</h3>
                    <input type="file" name="image1" accept="image/*" required>
                </div>

                <div class="upload-box">
                    <h3>📷 Image 2</h3>
                    <input type="file" name="image2" accept="image/*" required>
                </div>
            </div>

            <button type="submit">Compare Images</button>
        </form>

        {% if result %}
        <div class="result">
            <h2>Result</h2>
            {% for key, value in result.items() %}
            <div class="result-item">
                <span class="result-label">{{ key }}:</span>
                <span class="result-value">{{ value }}</span>
            </div>
            {% endfor %}
        </div>
        {% endif %}
    </div>
</body>
</html>
'''


In [6]:

@app.route('/', methods=['GET', 'POST'])
def home():
    result = None

    if request.method == 'POST':
        try:
            # Get uploaded images
            image1_file = request.files['image1']
            image2_file = request.files['image2']

            # Open images
            image1 = Image.open(image1_file)
            image2 = Image.open(image2_file)

            # Convert to RGB if needed
            if image1.mode != 'RGB':
                image1 = image1.convert('RGB')
            if image2.mode != 'RGB':
                image2 = image2.convert('RGB')

            # Process with your algorithm
            result = my_algorithm(image1, image2)

        except Exception as e:
            result = {
                "Error": str(e)
            }

    return render_template_string(HTML, result=result)



# RUN APP WITH NGROK

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print("="*60)
print("Your app is running!")
print(f"Open this URL: {public_url}")
print("="*60)

# Run Flask app
app.run(port=5000)

Your app is running!
Open this URL: NgrokTunnel: "https://a9b4c157bc6f.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 05:17:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 05:17:21] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 05:17:51] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 05:21:26] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2025 05:22:14] "POST / HTTP/1.1" 200 -
